In [155]:
#Import files module
import pandas as pd
import csv
import re                     ##Regular expressions
import glob
import os
import logging
import numpy as np
import datetime
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [156]:
#logging mechanism for debugging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [157]:
#Mapping months to numeric values
months = {'01':'Jan','02':'Feb','03':'Mar','04':'Apr','05':'May','06':'June','07':'July','08':'Aug','09':'Sept','10':'Oct','11':'Nov','12':'Dec'}

In [158]:
import pandas as pd

#Importing files from specific folders and list of holidays
bus_no='6080'                                                #Route number - Change this value accordingly 
path = r'D:/Dharmit/Final Year Project/Clean data Vikhroli/'+bus_no+'/Up'   #Up Direction files folder Path
down_path = r'D:/Dharmit/Final Year Project/Clean data Vikhroli/'+bus_no+'/Down' #Down Direction files folder path
allfiles = glob.glob(path + "/*.csv")
down_allfiles = glob.glob(down_path + "/*.csv")
Holiday_colnames = ['Holiday', 'Date', 'Day', 'Month']
Holiday_data = pd.read_csv('D:\Dharmit\Final Year Project\List of holidays.csv', names=Holiday_colnames)   #Holiday list for 2018
Holiday_2017 = pd.read_csv('D:\Dharmit\Final Year Project\List of holidays 2017.csv', names=Holiday_colnames)  #Holiday list for 2017
feature_names =  ['Time Slot','Trips','Passengers','Day','isHoliday']

# Up Direction Processing

In [159]:
#Making separate dataframes for each separate timeslot over many days
df1=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df2=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df3=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df4=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df5=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df6=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df7=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df8=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df9=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df10=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df11=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df12=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df13=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df14=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df15=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df16=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df17=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df18=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df19=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df20=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20]

#Dicing only requird data from list of holidays
holiday_day = Holiday_data['Day']
holiday_month = Holiday_data['Month']
holiday_day_2017 = Holiday_2017['Day']
holiday_month_2017 = Holiday_2017['Month']

In [160]:
Timeslot_values= ['00:00:01-05:00:00','05:00:01-06:00:00','06:00:01-07:00:00','07:00:01-08:00:00','08:00:01-09:00:00','09:00:01-10:00:00','10:00:01-11:00:00','11:00:01-12:00:00','12:00:01-13:00:00','13:00:01-14:00:00','14:00:01-15:00:00','15:00:01-16:00:00','16:00:01-17:00:00','17:00:01-18:00:00','18:00:01-19:00:00','19:00:01-20:00:00','20:00:01-21:00:00','21:00:01-22:00:00','22:00:01-23:00:00','23:00:01-24:00:00']
Days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}

#Obtaining the day and date details for current day 
now = datetime.datetime.now()   
current_month = months[str(now.month).zfill(2)] 
current_day = now.day
isCurrDay_holiday = 0
current_dow = Days[now.strftime("%A")]           #Obtaining current day of the week
for i in range(1,23):                            #checking if there is holiday today
    if(holiday_day[i]==current_day and current_month==holiday_month[i]):
        isCurrDay_holiday=1

for files in allfiles:
    #Obtaining Day and date details from filename
    base = os.path.basename(files)
    day = os.path.splitext(base)[0][29:]
    date = os.path.splitext(base)[0][26:28]

    month = os.path.splitext(base)[0][23:25]
    year = os.path.splitext(base)[0][18:22]
    #Determining whether holiday or not
    holiday = 0
    if year=='2017':
        for i in range(1,22):
            if(str(holiday_day_2017[i]).zfill(2)==date and months[month]==holiday_month_2017[i]):
            #print'Holiday 2017'
                holiday=1
    else:
        for i in range(1,23):
            if(holiday_day[i]==date and months[month]==holiday_month[i]):
                holiday=1
            
    #Pre-processing data into required format
    day = Days[day]                                        #Mapping day to numeric value
    with open(files,'rb') as f:
        timeslot = []
        trips = []
        passengers = []
        reader = csv.reader(f, delimiter=",")
        rows = [r for r in reader]
        for x in rows[1:]:
            timeslot.append(x[1])
            trips.append(x[2])
            if(x[3]==''):
                x[3]=0
            passengers.append(x[3])
 
    #Adding data into the dataframes created previously according to time slots
    for j in xrange(len(timeslot)):
        df[j]= df[j].append(pd.Series([timeslot[j],trips[j],int(passengers[j]),day,holiday], index = ['Time Slot','Trips','Passengers','Day','isHoliday']), ignore_index=True)
        df[j]=df[j].replace('',0)                      #Replacing NAN values by 0

In [161]:
#Linear Regression module to predict passengers and number of trips
import sklearn
import numpy as np
#import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
scale = StandardScaler()                            

err_sum=0   
# To obtain average error measure across all dataframes
for j in xrange(len(timeslot)):
    print 'Timeslot',j+1,':'
    pass_data=np.asarray(df[j][['Day','isHoliday']])
    pass_labels=np.asarray(df[j]['Passengers'])
    target=[[current_dow,isCurrDay_holiday]]                      # Example target list to check output format and debugging

    passenger_predictor = LinearRegression(normalize=True)          #Linear regression model to predict passenger frequency
    passenger_predictor.fit(pass_data, pass_labels)
    #err_measure = passenger_predictor.score(pass_data, pass_labels, sample_weight=None)  #Error in passenger frequency prediction
    pass_pred=passenger_predictor.predict(target)
    target=np.append(target,pass_pred)
    print target 
    print passenger_predictor.coef_
    print passenger_predictor.intercept_

Timeslot 1 :
[ 4.          0.          9.28664949]
[ 0.89704495 -1.28664949]
5.69846968225
Timeslot 2 :
[ 4.          0.          3.81425012]
[ 0.74133829 -3.81425012]
0.848896960482
Timeslot 3 :
[  4.           0.          15.20103898]
[ 0.24432994 -9.53437232]
14.2237192212
Timeslot 4 :
[  4.           0.          31.90822317]
[ -0.58247807 -13.2415565 ]
34.238135462
Timeslot 5 :
[  4.           0.          32.31710098]
[  2.19630865 -11.87265654]
23.5318663682
Timeslot 6 :
[  4.           0.          36.44063859]
[  3.80058003 -18.44063859]
21.2383184807
Timeslot 7 :
[  4.           0.          23.29194044]
[ 0.6135843  -4.51416266]
20.8376032311
Timeslot 8 :
[  4.           0.          30.93405269]
[ 0.48606004 -4.0451638 ]
28.9898125264
Timeslot 9 :
[  4.           0.          33.71529327]
[ 0.99763397  4.84026229]
29.7247573693
Timeslot 10 :
[  4.           0.          33.01748146]
[ 1.5187775   9.31585187]
26.942371451
Timeslot 11 :
[  4.           0.          32.97703297]
[-0.2

In [162]:
#Linear Regression module to predict passengers and number of trips
import sklearn
import numpy as np
#import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
scale = StandardScaler() 

up_pass=[]
up_trips=[]

err_sum=0   
# To obtain average error measure across all dataframes
print len(timeslot)
for j in xrange(len(timeslot)):
    print 'Timeslot',j+1,':'
    pass_data=np.asarray(df[j][['Day','isHoliday']])
    pass_labels=np.asarray(df[j]['Passengers'])
    target=[[current_dow,isCurrDay_holiday]]                      # Example target list to check output format and debugging

    passenger_predictor = LinearRegression(normalize=True)          #Linear regression model to predict passenger frequency
    passenger_predictor.fit(pass_data, pass_labels)
    #err_measure = passenger_predictor.score(pass_data, pass_labels, sample_weight=None)  #Error in passenger frequency prediction
    pass_pred=passenger_predictor.predict(target)
    up_pass.append(round(pass_pred))
    target=np.append(target,pass_pred)
    print target                                                    # Target features for predicting number of trips

    trip_data=np.asarray(df[j][['Day','isHoliday','Passengers']])   
    trip_labels=np.asarray(df[j]['Trips'])

    #generate a model of polynomial features
    poly = PolynomialFeatures(5)
    target=[target]

    #transform the x data for proper fitting (for single variable type it returns,[1,x,x**2])
    trip_data_= poly.fit_transform(trip_data)

    #transform the prediction to fit the model type
    target_ = poly.fit_transform(target)
    
    #here we can remove polynomial orders we don't want
    #for instance I'm removing the `x` component
    trip_data_ = np.delete(trip_data_,[7,10,13],axis=1)
    target_ = np.delete(target_,[7,10,13],axis=1)
    
    #generate the regression object
    clf = linear_model.LinearRegression()
    #preform the actual regression
    clf.fit(trip_data_, trip_labels)
    trips=round(clf.predict(target_))
    if trips <0:
        trips=0
    if trips > 10:
        trips=5
    up_trips.append(trips)
    print "Prediction = ",trips
    err_sum+=clf.score(trip_data_, trip_labels, sample_weight=None)
    #print clf.coef_
    
print err_sum/len(timeslot)

20
Timeslot 1 :
[ 4.          0.          9.28664949]
Prediction =  -0.0
Timeslot 2 :
[ 4.          0.          3.81425012]
Prediction =  6.0
Timeslot 3 :
[  4.           0.          15.20103898]
Prediction =  5.0
Timeslot 4 :
[  4.           0.          31.90822317]
Prediction =  6.0
Timeslot 5 :
[  4.           0.          32.31710098]
Prediction =  5.0
Timeslot 6 :
[  4.           0.          36.44063859]
Prediction =  6.0
Timeslot 7 :
[  4.           0.          23.29194044]
Prediction =  4.0
Timeslot 8 :
[  4.           0.          30.93405269]
Prediction =  3.0
Timeslot 9 :
[  4.           0.          33.71529327]
Prediction =  3.0
Timeslot 10 :
[  4.           0.          33.01748146]
Prediction =  3.0
Timeslot 11 :
[  4.           0.          32.97703297]
Prediction =  3.0
Timeslot 12 :
[  4.           0.          47.25758032]
Prediction =  3.0
Timeslot 13 :
[  4.           0.          58.30291157]
Prediction =  3.0
Timeslot 14 :
[  4.           0.          80.58773886]
Predict

# For DOWN Direction

In [164]:
#Making separate dataframes for each separate timeslot over many days
df1=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df2=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df3=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df4=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df5=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df6=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df7=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df8=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df9=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df10=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df11=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df12=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df13=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df14=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df15=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df16=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df17=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df18=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df19=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df20=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20]

In [166]:
Timeslot_values= ['00:00:01-05:00:00','05:00:01-06:00:00','06:00:01-07:00:00','07:00:01-08:00:00','08:00:01-09:00:00','09:00:01-10:00:00','10:00:01-11:00:00','11:00:01-12:00:00','12:00:01-13:00:00','13:00:01-14:00:00','14:00:01-15:00:00','15:00:01-16:00:00','16:00:01-17:00:00','17:00:01-18:00:00','18:00:01-19:00:00','19:00:01-20:00:00','20:00:01-21:00:00','21:00:01-22:00:00','22:00:01-23:00:00','23:00:01-24:00:00']
Days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}

#Obtaining the day and date details for current day 
now = datetime.datetime.now()   
current_month = months[str(now.month).zfill(2)] 
current_day = now.day
isCurrDay_holiday = 0
current_dow = Days[now.strftime("%A")]           #Obtaining current day of the week
for i in range(1,24):                            #checking if there is holiday today
    if(holiday_day[i]==current_day and current_month==holiday_month[i]):
        isCurrDay_holiday=1

for files in down_allfiles:
    #Obtaining Day and date details from filename
    base = os.path.basename(files)
    day = os.path.splitext(base)[0][31:]
    date = os.path.splitext(base)[0][29:31]
    month = os.path.splitext(base)[0][26:28]
    
    #Determining whether holiday or not
    year = os.path.splitext(base)[0][21:25]
    #Determining whether holiday or not
    holiday = 0
    if year=='2017':
        for i in range(1,22):
            if(str(holiday_day_2017[i]).zfill(2)==date and months[month]==holiday_month_2017[i]):
            #print'Holiday 2017'
                holiday=1
    else:
        for i in range(1,23):
            if(holiday_day[i]==date and months[month]==holiday_month[i]):
                holiday=1
            
    #Pre-processing data into required format
    day = Days[day]                                        #Mapping day to numeric value
    with open(files,'rb') as f:
        timeslot = []
        trips = []
        passengers = []
        reader = csv.reader(f, delimiter=",")
        rows = [r for r in reader]
        for x in rows[1:]:
            timeslot.append(x[1])
            trips.append(x[2])
            if(x[3]==''):
                x[3]=0
            passengers.append(x[3])
 
    #Adding data into the dataframes created previously according to time slots
    for j in xrange(len(timeslot)):
        df[j]= df[j].append(pd.Series([timeslot[j],trips[j],int(passengers[j]),day,holiday], index = ['Time Slot','Trips','Passengers','Day','isHoliday']), ignore_index=True)
        df[j]=df[j].replace('',0)                      #Replacing NAN values by 0

In [167]:
down_trips=[]
down_pass=[]
#Linear Regression module to predict passengers and number of trips
import sklearn
import numpy as np
#import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
scale = StandardScaler() 

err_sum=0   
# To obtain average error measure across all dataframes
print len(timeslot)
for j in xrange(len(timeslot)):
    print 'Timeslot',j+1,':'
    pass_data=np.asarray(df[j][['Day','isHoliday']])
    pass_labels=np.asarray(df[j]['Passengers'])
    target=[[current_dow,isCurrDay_holiday]]                      # Example target list to check output format and debugging

    passenger_predictor = LinearRegression(normalize=True)          #Linear regression model to predict passenger frequency
    passenger_predictor.fit(pass_data, pass_labels)
    #err_measure = passenger_predictor.score(pass_data, pass_labels, sample_weight=None)  #Error in passenger frequency prediction
    pass_pred=passenger_predictor.predict(target)
    down_pass.append(round(pass_pred))
    target=np.append(target,pass_pred)
    print target                                                    # Target features for predicting number of trips

    trip_data=np.asarray(df[j][['Day','isHoliday','Passengers']])   
    trip_labels=np.asarray(df[j]['Trips'])

    #generate a model of polynomial features
    poly = PolynomialFeatures(5)
    target=[target]

    #transform the x data for proper fitting (for single variable type it returns,[1,x,x**2])
    trip_data_= poly.fit_transform(trip_data)

    #transform the prediction to fit the model type
    target_ = poly.fit_transform(target)
    
    #here we can remove polynomial orders we don't want
    #for instance I'm removing the `x` component
    trip_data_ = np.delete(trip_data_,[7,10,13],axis=1)
    target_ = np.delete(target_,[7,10,13],axis=1)
    
    #generate the regression object
    clf = linear_model.LinearRegression()
    #preform the actual regression
    clf.fit(trip_data_, trip_labels)
    trips=round(clf.predict(target_))
    if trips <0:
        trips=0
    if trips > 10:
        trips=5
    down_trips.append(trips)
    print "Prediction = ",trips
    err_sum+=clf.score(trip_data_, trip_labels, sample_weight=None)
    #print clf.coef_
    
print err_sum/len(timeslot)

20
Timeslot 1 :
[  4.          0.         10.9823613]
Prediction =  2.0
Timeslot 2 :
[  4.           0.          20.54869387]
Prediction =  2.0
Timeslot 3 :
[  4.          0.         79.1842396]
Prediction =  6.0
Timeslot 4 :
[   4.            0.          127.50339358]
Prediction =  6.0
Timeslot 5 :
[   4.            0.          127.22216084]
Prediction =  5.0
Timeslot 6 :
[   4.            0.          119.08882488]
Prediction =  4.0
Timeslot 7 :
[  4.           0.          79.78257695]
Prediction =  3.0
Timeslot 8 :
[  4.           0.          65.21264884]
Prediction =  4.0
Timeslot 9 :
[  4.           0.          47.68978881]
Prediction =  3.0
Timeslot 10 :
[  4.           0.          36.63805886]
Prediction =  3.0
Timeslot 11 :
[  4.           0.          29.98388003]
Prediction =  3.0
Timeslot 12 :
[  4.           0.          23.03808133]
Prediction =  3.0
Timeslot 13 :
[  4.           0.          25.11109863]
Prediction =  3.0
Timeslot 14 :
[  4.           0.          29.00724556]

In [168]:
#Saving Results into CSV files 
forecasted_trips = pd.DataFrame(columns=['Time Slot','Up_Trips','Up_pass','Down_Trips','Down_pass'])
save_path=r'D:/Dharmit/Final Year Project/Forecasted Trips/'
for j in xrange(len(timeslot)):
    forecasted_trips= forecasted_trips.append(pd.Series([timeslot[j],up_trips[j],up_pass[j],down_trips[j],down_pass[j]], index = ['Time Slot','Up_Trips','Up_pass','Down_Trips','Down_pass']), ignore_index=True)
filename ='Forecasted_trips_'+bus_no+'_'+str(now.month).zfill(2) +'_'+str(now.day)+'_'+str(day)+'_Poly'
forecasted_trips.to_csv(save_path + filename +'.csv')